In [ ]:
import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt
import pandas as pd

sns.set(font_scale=2., style='whitegrid')

In [ ]:
import wandb
from tqdm.auto import tqdm
import pandas as pd

def get_summary_metrics(sweep_id, filter_func=None):
  api = wandb.Api(timeout=60)
  sweep = api.sweep(sweep_id)

  data = []
  for run in tqdm(sweep.runs, desc='Runs', leave=False):
    if callable(filter_func) and not filter_func(run):
      continue
    
    run_summary = dict(run_id=run.id, **run.config, **run.summary)

    data.append(run_summary)

  return sweep, pd.DataFrame(data)

In [ ]:
# _, results = get_summary_metrics('deeplearn/fspace-inference/mwzafu81')
# metrics = results[['run_id', 'seed', 'dataset', 'ctx_dataset', 'label_noise', 'reg_scale', 's/test/acc', 's/test/sel_acc', 's/test/avg_nll', 's/test/ece']]
# metrics['s/test/acc'] *= 100
# metrics['s/test/ece'] *= 100
# # metrics.to_csv('c10_lnoise.csv', index=False)

In [ ]:
metrics = pd.read_csv('results/c10_lnoise.csv')

fig, ax = plt.subplots(figsize=(7,5))

sns.barplot(data=metrics[metrics.ctx_dataset == 'cifar100'], ax=ax, x='label_noise', y='s/test/acc',
             hue='reg_scale', errorbar='sd', hue_order=[0.000001, 0.000000],
             palette=sns.color_palette('Set2', 2))

ax.set(ylabel='Accuracy', xlabel='Label Noise')
# ax.set_xticks([0., .2, .4, .6, .8])
# ax.set_yticks([35, 55, 75, 95])

def labelmap(v):
    return 'L-MAP' if float(v) > 0. else 'PS-MAP'

handles, labels = ax.get_legend_handles_labels()
labels = [labelmap(l) for l in labels]
ax.legend(handles=handles, labels=labels, fontsize=20, title='Method', loc='best')

fig.show()
# fig.savefig('lmap_c10_lnoise.pdf', bbox_inches='tight')